In [70]:
import warnings
import pandas as pd
import pyodbc
import numpy as np

warnings.simplefilter('ignore')

# Connection details for the source SQL Server database
source_db = {
    'servername': r'DESKTOP-R9U7MMK\SQLEXPRESS',
    'database': 'AdventureWorks2019'
}

source_db_northwind = {
    'servername': r'DESKTOP-R9U7MMK\SQLEXPRESS',
    'database': 'NORTHWIND'
}

# Connection details for the destination SQL Server database
destination_db = {
    'servername': r'DESKTOP-R9U7MMK\SQLEXPRESS',
    'database': 'DWUnitedOutdoors'
}

# Connection string for the source SQL Server database
source_conn_str = (
    'Driver={SQL Server};'
    'SERVER=' + source_db['servername'] + ';'
    'DATABASE=' + source_db['database'] + ';'
    'Trusted_Connection=yes;'
)

# Connection string for the source SQL Server database
source_conn_str_northwind = (
    'Driver={SQL Server};'
    'SERVER=' + source_db_northwind['servername'] + ';'
    'DATABASE=' + source_db_northwind['database'] + ';'
    'Trusted_Connection=yes;'
)

# Connection string for the destination SQL Server database
destination_conn_str = (
    'Driver={SQL Server};'
    'SERVER=' + destination_db['servername'] + ';'
    'DATABASE=' + destination_db['database'] + ';'
    'Trusted_Connection=yes;'
)

In [37]:
#Person.Person
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        person_df = pd.read_sql_query("SELECT * FROM Person.person", source_conn)
        personPhone_df = pd.read_sql_query("SELECT PhoneNumber, PhoneNumberTypeID, BusinessEntityID FROM Person.PersonPhone", source_conn)
        phoneNumberType_df = pd.read_sql_query("SELECT PhoneNumberTypeID, Name FROM Person.PhoneNumberType", source_conn)
        personEmailAdress_df = pd.read_sql_query("SELECT EmailAddressID, EmailAddress, BusinessEntityID FROM Person.EmailAddress", source_conn)
        personPassword_df = pd.read_sql_query("SELECT PasswordHash, PasswordSalt, BusinessEntityID FROM Person.Password", source_conn)

        # Merging DataFrames
        person_df = pd.merge(person_df, personPhone_df, on='BusinessEntityID', how='left')
        person_df = pd.merge(person_df, phoneNumberType_df, on='PhoneNumberTypeID', how='left')
        person_df = pd.merge(person_df, personEmailAdress_df, on='BusinessEntityID', how='left')
        person_df = pd.merge(person_df, personPassword_df, on='BusinessEntityID', how='left')

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Person]
        (BusinessEntityID, PersonType, NameStyle, Title, FirstName, MiddleName, LastName, Suffix, EmailPromotion, PasswordSalt, PasswordHash, EmailAddress, Name, EmailAddressID, PhoneNumberTypeID)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = person_df[['BusinessEntityID', 'PersonType', 'NameStyle', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'EmailPromotion', 'PasswordSalt', 'PasswordHash', 'EmailAddress', 'Name', 'EmailAddressID', 'PhoneNumberTypeID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [22]:
#businessEntity_df
# Establish connection to the source SQL Server database and read
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        businessEntity_df = pd.read_sql_query("SELECT BusinessEntityID FROM Person.BusinessEntity", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[BusinessEntity]
        (BusinessEntityID)
        VALUES (?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = businessEntity_df[['BusinessEntityID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)
# businessEntity_df

Data inserted successfully.


In [55]:
#businessEntityAddress_df
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        businessEntityAddress_df = pd.read_sql_query("SELECT AddressID, AddressTypeID, BusinessEntityID FROM Person.BusinessEntityAddress", source_conn)
        addressType_df = pd.read_sql_query("SELECT AddressTypeID, Name FROM Person.AddressType", source_conn)
                                           
        # Merging DataFrames
        businessEntityAddress_df = pd.merge(businessEntityAddress_df, addressType_df, on='AddressTypeID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[BusinessEntityAddress]
        (BusinessEntityID, AddressID, AddressTypeID, Name)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = businessEntityAddress_df[['BusinessEntityID', 'AddressID', 'AddressTypeID', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Error while connecting to the destination SQL Server or inserting data: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_BusinessEntityAddress'. Cannot insert duplicate key in object 'dbo.BusinessEntityAddress'. The duplicate key value is (2). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [15]:
#person.Address_df
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        personAddress_df = pd.read_sql_query("SELECT AddressID, AddressLine1, AddressLine2, City, StateProvinceID, PostalCode, SpatialLocation FROM Person.Address", source_conn)
                                           
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Address]
        (AddressID, AddressLine1, AddressLine2, City, StateProvinceID, PostalCode, SpatialLocation)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = personAddress_df[['AddressID', 'AddressLine1', 'AddressLine2', 'City', 'StateProvinceID', 'PostalCode', 'SpatialLocation']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [86]:
#Sales.CreditCard
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        creditCard_df = pd.read_sql_query("SELECT CreditCardID, CardType, CardNumber, ExpMonth, ExpYear FROM Sales.CreditCard", source_conn)
        personCreditCard_df = pd.read_sql_query("SELECT CreditCardID, BusinessEntityID FROM Sales.PersonCreditCard", source_conn)
        
        # Merging DataFrames
        creditCard_df = pd.merge(creditCard_df, personCreditCard_df, on='CreditCardID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

#Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[CreditCard]
        (CreditCardID, BusinessEntityID, CardType, CardNumber, ExpMonth, ExpYear)
        VALUES (?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = creditCard_df[['CreditCardID', 'BusinessEntityID', 'CardType', 'CardNumber', 'ExpMonth', 'ExpYear']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [85]:
#Sales.SalesPerson
# Establish connection to the source SQL Server database and read data  
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesPerson_df = pd.read_sql_query("SELECT BusinessEntityID, TerritoryID, SalesQuota, Bonus, CommissionPct, SalesLastYear, SalesYTD FROM Sales.SalesPerson", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Fill NaN values in selected columns
salesPerson_df['TerritoryID'].fillna(0.0, inplace=True)
salesPerson_df['SalesQuota'].fillna(0.0, inplace=True)
salesPerson_df['CommissionPct'].fillna(0.0, inplace=True)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
         
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesPerson]
        (BusinessEntityID, TerritoryID, SalesQuota, Bonus, CommisionPct, SalesLastYear, SalesYTD)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """
         
        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesPerson_df[['BusinessEntityID', 'TerritoryID', 'SalesQuota', 'Bonus', 'CommissionPct', 'SalesLastYear', 'SalesYTD']].values.tolist()
         
        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)
         
        # Commit transaction
        destination_conn.commit()
         
        print("Data inserted successfully into SalesPerson table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Error while connecting to the destination SQL Server: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot insert the value NULL into column 'SalesPersonID', table 'DWUnitedOutdoors.dbo.SalesPerson'; column does not allow nulls. INSERT fails. (515) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [57]:
#Sales.Store
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesStore_df = pd.read_sql_query("SELECT BusinessEntityID, Name, SalesPersonID FROM Sales.Store", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for salesPersonQuotaHistory_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Store]
        (BusinessEntityID, Name, SalesPersonID)
        VALUES (?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesStore_df[['BusinessEntityID', 'Name', 'SalesPersonID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into salesstore table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into SalesPersonQuotaHistory table.


In [59]:
#Sales.SalesPersonQuotaHistory
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesPersonQuotaHistory_df = pd.read_sql_query("SELECT BusinessEntityID, QuotaDate, SalesQuota FROM Sales.SalesPersonQuotaHistory", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for salesPersonQuotaHistory_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesPersonQuotaHistory]
        (BusinessEntityID, QuotaDate, SalesQuota)
        VALUES (?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesPersonQuotaHistory_df[['BusinessEntityID', 'QuotaDate', 'SalesQuota']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into SalesPersonQuotaHistory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Error while connecting to the destination SQL Server: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_SalesPersonQuotaHistory'. Cannot insert duplicate key in object 'dbo.SalesPersonQuotaHistory'. The duplicate key value is (May 31 2011 12:00AM). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [61]:
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesTerritoryHistory_df = pd.read_sql_query("SELECT BusinessEntityID, TerritoryID, StartDate, EndDate FROM Sales.salesTerritoryHistory", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Convert date columns to datetime objects
date_columns = ['StartDate', 'EndDate']
for column in date_columns:
    salesTerritoryHistory_df[column] = pd.to_datetime(salesTerritoryHistory_df[column], errors='coerce')

# Drop rows with invalid dates
salesTerritoryHistory_df.dropna(subset=date_columns, inplace=True)

# Format dates as strings in 'YYYY-MM-DD' format
salesTerritoryHistory_df['StartDate'] = salesTerritoryHistory_df['StartDate'].dt.strftime('%Y-%m-%d')
salesTerritoryHistory_df['EndDate'] = salesTerritoryHistory_df['EndDate'].dt.strftime('%Y-%m-%d')

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for salesTerritoryHistory_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesTerritoryHistory]
        (BusinessEntityID, TerritoryID, StartDate, EndDate)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesTerritoryHistory_df[['BusinessEntityID', 'TerritoryID', 'StartDate', 'EndDate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into SalesTerritoryHistory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into SalesTerritoryHistory table.


In [63]:
#Sales.SalesTerritory
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        SalesTerritory_df = pd.read_sql_query("SELECT TerritoryID, Name, CountryRegionCode, [Group], SalesYTD, SalesLastYear, CostYTD, CostLastYear FROM Sales.SalesTerritory", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for SalesTerritory_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesTerritory]
        (TerritoryID, Name, CountryRegionCode, [Group], SalesYTD, SalesLastYear, CostYTD, CostLastYear)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = SalesTerritory_df[['TerritoryID', 'Name', 'CountryRegionCode', 'Group', 'SalesYTD', 'SalesLastYear', 'CostYTD', 'CostLastYear']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into SalesTerritory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into SalesTerritory table.


In [64]:
#Sales.CountryRegionCurrency
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        CountryRegionCurrency_df = pd.read_sql_query("SELECT CountryRegionCode, CurrencyCode FROM Sales.CountryRegionCurrency",source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for CountryRegionCurrency_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[CountryRegionCurrency]
        (CountryRegionCode, CurrencyCode)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = CountryRegionCurrency_df[['CountryRegionCode', 'CurrencyCode']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into CountryRegionCurrency table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Error while connecting to the destination SQL Server: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_CountryRegionCurrency'. Cannot insert duplicate key in object 'dbo.CountryRegionCurrency'. The duplicate key value is (AT). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [87]:
#Sales.Currency
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        currency_df = pd.read_sql_query("SELECT CurrencyCode, Name FROM Sales.Currency",source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for currency_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Currency]
        (CurrencyCode, Name)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = currency_df[['CurrencyCode', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into Currency table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into Currency table.


In [93]:
#Sales.order
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        sales_order_header_df = pd.read_sql_query("SELECT SalesOrderID, OrderDate, DueDate, ShipDate, Status, SalesOrderNumber, PurchaseOrderNumber, AccountNumber, CustomerID, SalesPersonID, TerritoryID, BillToAddressID, ShipToAddressID, ShipMethodID, CurrencyRateID, CreditCardID, SubTotal, TaxAmt, Freight, TotalDue   FROM Sales.SalesOrderHeader", source_conn)
        sales_order_detail_df = pd.read_sql_query("SELECT SalesOrderDetailID, CarrierTrackingNumber , OrderQty, ProductID,SpecialOfferID,UnitPrice, UnitPriceDiscount, LineTotal, SalesOrderID  FROM Sales.SalesOrderDetail ", source_conn)
        currency_rate_df = pd.read_sql_query("SELECT CurrencyRateID, CurrencyRateDate,FromCurrencyCode, ToCurrencyCode, AverageRate, EndOfDayRate  FROM Sales.CurrencyRate", source_conn)
        
        ## merging dataframes
        sales_order_df = pd.merge(sales_order_header_df, currency_rate_df, on='CurrencyRateID', how='left')
        sales_order_df = pd.merge(sales_order_detail_df, sales_order_df, on='SalesOrderID', how='left')
                           
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    
sales_order_df['OrderDate'] = pd.to_datetime(sales_order_df['OrderDate'], errors='coerce')
sales_order_df['DueDate'] = pd.to_datetime(sales_order_df['DueDate'], errors='coerce')
sales_order_df['ShipDate'] = pd.to_datetime(sales_order_df['ShipDate'], errors='coerce')
sales_order_df['CurrencyRateDate'] = pd.to_datetime(sales_order_df['CurrencyRateDate'], errors='coerce')

# Replace NaT values with a default datetime value
default_date = pd.Timestamp('1900-01-01')  # or any other default value you prefer
sales_order_df['OrderDate'].fillna(default_date, inplace=True)
sales_order_df['DueDate'].fillna(default_date, inplace=True)
sales_order_df['ShipDate'].fillna(default_date, inplace=True)
sales_order_df['CurrencyRateDate'].fillna(default_date, inplace=True)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for sales_order_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesOrder]
        (SalesOrderID, OrderDate, DueDate, ShipDate, Status, SalesOrderNumber, PurchaseOrderNumber, AccountNumber, CustomerID, SalesPersonID, TerritoryID, BillToAddressID, ShipToAddressID, ShipMethodID, CurrencyRateID, CreditCardID, SubTotal, TaxAmt, Freight, TotalDue, SalesOrderDetailID, CarrierTrackingNumber, OrderQty, ProductID, SpecialOfferID, UnitPrice, UnitPriceDiscount, LineTotal, CurrencyRateDate, FromCurrencyCode, ToCurrencyCode, AverageRate, EndOfDayRate)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = sales_order_df[['SalesOrderID', 'OrderDate', 'DueDate', 'ShipDate', 'Status', 'SalesOrderNumber', 'PurchaseOrderNumber', 'AccountNumber', 'CustomerID', 'SalesPersonID', 'TerritoryID', 'BillToAddressID', 'ShipToAddressID', 'ShipMethodID', 'CurrencyRateID', 'CreditCardID', 'SubTotal', 'TaxAmt', 'Freight', 'TotalDue', 'SalesOrderDetailID', 'CarrierTrackingNumber', 'OrderQty', 'ProductID', 'SpecialOfferID', 'UnitPrice', 'UnitPriceDiscount', 'LineTotal', 'CurrencyRateDate', 'FromCurrencyCode', 'ToCurrencyCode', 'AverageRate', 'EndOfDayRate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into SalesOrder table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)
sales_order_df

Error while connecting to the destination SQL Server: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 18 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')


,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,SalesOrderID,OrderDate,...,CreditCardID,SubTotal,TaxAmt,Freight,TotalDue,CurrencyRateDate,FromCurrencyCode,ToCurrencyCode,AverageRate,EndOfDayRate
0,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,43659,2011-05-31,...,16281.0,20565.6206,1971.5149,616.0984,23153.2339,1900-01-01,NaN,NaN,NaN,NaN
1,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,43659,2011-05-31,...,16281.0,20565.6206,1971.5149,616.0984,23153.2339,1900-01-01,NaN,NaN,NaN,NaN
2,3,4911-403C-98,1,778,1,2024.994,0.0,2024.994,43659,2011-05-31,...,16281.0,20565.6206,1971.5149,616.0984,23153.2339,1900-01-01,NaN,NaN,NaN,NaN
3,4,4911-403C-98,1,771,1,2039.994,0.0,2039.994,43659,2011-05-31,...,16281.0,20565.6206,1971.5149,616.0984,23153.2339,1900-01-01,NaN,NaN,NaN,NaN
4,5,4911-403C-98,1,772,1,2039.994,0.0,2039.994,43659,2011-05-31,...,16281.0,20565.6206,1971.5149,616.0984,23153.2339,1900-01-01,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121312,121313,None,1,878,1,21.980,0.0,21.980,75122,2014-06-30,...,18719.0,30.9700,2.4776,0.7743,34.2219,1900-01-01,NaN,NaN,NaN,NaN
121313,121314,None,1,712,1,8.990,0.0,8.990,75122,2014-06-30,...,18719.0,30.9700,2.4776,0.7743,34.2219,1900-01-01,NaN,NaN,NaN,NaN
121314,121315,None,1,878,1,21.980,0.0,21.980,75123,2014-06-30,...,10084.0,189.9700,15.1976,4.7493,209.9169,1900-01-01,NaN,NaN,NaN,NaN
121315,121316,None,1,879,1,159.000,0.0,159.000,75123,2014-06-30,...,10084.0,189.9700,15.1976,4.7493,209.9169,1900-01-01,NaN,NaN,NaN,NaN


In [94]:
#Sales.ship
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ship_method_df = pd.read_sql_query("SELECT ShipMethodID, Name, ShipBase, ShipRate FROM Purchasing.ShipMethod", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for ship_method
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ShipMethod]
        (ShipMethodID, Name, ShipBase, ShipRate)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = ship_method_df[['ShipMethodID', 'Name', 'ShipBase', 'ShipRate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into ShipMethod table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into ShipMethod table.


In [95]:
#Sales.salesOrderHeaderSalesReason
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesOrderHeaderSalesReason_df = pd.read_sql_query("SELECT SalesOrderID, SalesReasonID FROM Sales.SalesOrderHeaderSalesReason", source_conn)
        salesReason_df = pd.read_sql_query("SELECT Name, ReasonType, SalesReasonID FROM Sales.SalesReason", source_conn)

        ## merging dataframes
        salesOrderHeaderSalesReason_df = pd.merge(salesOrderHeaderSalesReason_df, salesReason_df, on='SalesReasonID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for salesOrderHeaderSalesReason
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[salesOrderHeaderSalesReason]
        (SalesOrderID, SalesReasonID, Name, ReasonType)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesOrderHeaderSalesReason_df[['SalesOrderID', 'SalesReasonID', 'Name', 'ReasonType']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into salesOrderHeaderSalesReason table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into salesOrderHeaderSalesReason table.


In [98]:
#Person.CountryRegion
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        CountryRegion_df = pd.read_sql_query("SELECT CountryRegionCode, Name FROM Person.CountryRegion", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for CountryRegion
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[CounrtyRegion]
        (CountryRegionCode, Name)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = CountryRegion_df[['CountryRegionCode', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into CountryRegion table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into CountryRegion table.


In [102]:
#Person.StateProvince
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        StateProvince_df = pd.read_sql_query("SELECT StateProvinceID, StateProvinceCode, CountryRegionCode, Name, TerritoryID FROM Person.StateProvince", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for StateProvince
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[StateProvince]
        (StateProvinceID, StateProvinceCode, CountryRegionCode, Name, TerritoryID)
        VALUES (?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = StateProvince_df[['StateProvinceID', 'StateProvinceCode', 'CountryRegionCode', 'Name', 'TerritoryID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into StateProvince table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into StateProvince table.


In [104]:
#Person.BusinessEntityContact
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        BusinessEntityContact_df = pd.read_sql_query("SELECT BusinessEntityID, PersonID, ContactTypeID FROM Person.BusinessEntityContact", source_conn)
        ContactType_df = pd.read_sql_query("SELECT Name, ContactTypeID FROM Person.ContactType", source_conn)
        
        ## merging dataframes
        BusinessEntityContact_df = pd.merge(BusinessEntityContact_df, ContactType_df, on='ContactTypeID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for BusinessEntityContact
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[BusinessEntityContact]
        (BusinessEntityID, PersonID, ContactTypeID, Name)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = BusinessEntityContact_df[['BusinessEntityID', 'PersonID', 'ContactTypeID', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into BusinessEntityContact table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Error while connecting to the destination SQL Server: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_BusinessEntityContact'. Cannot insert duplicate key in object 'dbo.BusinessEntityContact'. The duplicate key value is (624). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [13]:
#Purchasing.PurchaseOrder
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        PurchaseOrderDetail_df = pd.read_sql_query("SELECT PurchaseOrderID, PurchaseOrderDetailID, DueDate, OrderQty, ProductID, UnitPrice, LineTotal, ReceivedQty, RejectedQty, StockedQty FROM Purchasing.PurchaseOrderDetail", source_conn)
        PurchaseOrderHeader_df = pd.read_sql_query("SELECT PurchaseOrderID, RevisionNumber, Status, EmployeeID, VendorID, ShipMethodID, OrderDate, ShipDate, Subtotal, taxAmt, Freight, TotalDue FROM Purchasing.PurchaseOrderHeader ", source_conn)
        
        ## merging dataframes
        PurchaseOrderDetail_df = pd.merge(PurchaseOrderHeader_df, PurchaseOrderDetail_df, on='PurchaseOrderID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    # Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[PurchaseOrder]
        (PurchaseOrderID, PurchaseOrderDetailID, DueDate, OrderQty, ProductID, UnitPrice, LineTotal, ReceivedQty, StockedQty, RejectedQty, RevisionNumber, Status, EmployeeID, VendorID, ShipMethodID, OrderDate, ShipDate, Subtotal, taxAmt, TotalDue)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? )
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = PurchaseOrderDetail_df[['PurchaseOrderID', 'PurchaseOrderDetailID', 'DueDate', 'OrderQty', 'ProductID', 'UnitPrice', 'LineTotal', 'ReceivedQty', 'StockedQty', 'RejectedQty', 'RevisionNumber', 'Status', 'EmployeeID', 'VendorID', 'ShipMethodID', 'ShipDate', 'Subtotal', 'taxAmt', 'TotalDue' ]].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()
        
        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

Error while connecting to the source SQL Server: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_PurchaseOrder'. Cannot insert duplicate key in object 'dbo.PurchaseOrder'. The duplicate key value is (2). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [105]:
#Purchasing.Vendor
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        Vendor_df = pd.read_sql_query("SELECT BusinessEntityID, AccountNumber, Name, CreditRating, PurchasingWebServiceURL FROM Purchasing.Vendor", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Vendor]
        (BusinessEntityID, AccountNumber, Name, CreditRating, PurchasingWebServiceURL)
        VALUES (?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = Vendor_df[['BusinessEntityID', 'AccountNumber', 'Name', 'CreditRating', 'PurchasingWebServiceURL']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()
        
        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully.


In [106]:
##Production.ProductSubCategory
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ProductSubCategory_df = pd.read_sql_query("SELECT ProductSubCategoryID ,ProductCategoryID, Name FROM Production.ProductSubCategory", source_conn)
        ProductCategory_df = pd.read_sql_query("SELECT Name, ProductCategoryID FROM Production.ProductCategory", source_conn)

        ## merging dataframes
        ProductSubCategory_df = pd.merge(ProductSubCategory_df, ProductCategory_df, on='ProductCategoryID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    # Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductSubCategory]
        (ProductSubCategoryID, ProductCategoryID, SubCategory_Name, Category_Name)
        VALUES (?, ?, ?, ?)
        """
        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = ProductSubCategory_df[['ProductSubCategoryID', 'ProductCategoryID', 'Name_x', 'Name_y']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Error while connecting to the destination SQL Server or inserting data: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_ProductSubCategory'. Cannot insert duplicate key in object 'dbo.ProductSubCategory'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [107]:
#Production.Product
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        productionproduct_df = pd.read_sql_query("SELECT ProductID ,Name, ProductNumber, Color, SafetyStockLevel, ReorderPoint, StandardCost, ListPrice, Size, SizeUnitMeasureCode, WeightUnitMeasureCode, Weight, DaysToManufacture, ProductLine, Class, Style, ProductSubcategoryID, SellStartDate, SellEndDate, DiscontinuedDate FROM Production.Product", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    # Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductionProduct]
        (ProductID, Name, ProductNumber, Color, SafetyStockLevel, ReorderPoint, StandardCost, ListPrice, Size, SizeUnitMeasureCode, WeightUnitMeasureCode, Weight, DaysToManufacture, ProductLine, Class, Style, ProductSubcategoryID, SellStartDate, SellEndDate, DiscontinuedDate)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = productionproduct_df[['ProductID', 'Name', 'ProductNumber', 'Color', 'SafetyStockLevel', 'ReorderPoint', 'StandardCost', 'ListPrice', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'Weight', 'DaysToManufacture', 'ProductLine', 'Class', 'Style', 'ProductSubcategoryID', 'SellStartDate', 'SellEndDate', 'DiscontinuedDate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)
#productionproduct_df

Error while connecting to the destination SQL Server or inserting data: ('22008', '[22008] [Microsoft][ODBC SQL Server Driver]Overloop in het datum-/tijdveld (0) (SQLExecDirectW)')


In [28]:
#Production.Work
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        work_order_df = pd.read_sql_query("SELECT WorkOrderID, ProductID, OrderQty, StockedQty, ScrappedQty, StartDate, EndDate, DueDate FROM Production.WorkOrder", source_conn)
        work_order_routing_df = pd.read_sql_query("SELECT WorkOrderID, OperationSequence, LocationID, ScheduledStartDate, ScheduledEndDate, ActualStartDate, ActualEndDate, ActualResourceHrs, PlannedCost, ActualCost FROM Production.WorkOrderRouting", source_conn)
         ## merging dataframes
        work_order_df = pd.merge(work_order_routing_df, work_order_df, on='WorkOrderID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
        # Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Work]
        (WorkOrderID, ProductID, OperationSequence, LocationID, ScheduledStartDate, ScheduledEndDate, ActualStartDate, ActualEndDate, ActualResourceHrs, PlannedCost, ActualCost, OrderQty, StockedQty, StartDate, EndDate, DueDate)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = work_order_df[['WorkOrderID', 'ProductID', 'OperationSequence', 'LocationID', 'ScheduledStartDate', 'ScheduledEndDate', 'ActualStartDate', 'ActualEndDate', 'ActualResourceHrs', 'PlannedCost', 'ActualCost', 'OrderQty', 'StockedQty', 'StartDate', 'EndDate', 'DueDate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Error while connecting to the destination SQL Server or inserting data: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_Work'. Cannot insert duplicate key in object 'dbo.Work'. The duplicate key value is (747). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [113]:
#Production.ProductionHistory
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ProductCostHistory_df = pd.read_sql_query("SELECT ProductID, StartDate, EndDate, StandardCost FROM Production.ProductCostHistory", source_conn)
        ProductListPriceHistory_df = pd.read_sql_query("SELECT ProductID, StartDate, EndDate, ListPrice FROM Production.ProductListPriceHistory", source_conn)
        TransactionHistory_df = pd.read_sql_query("SELECT ProductID, TransactionID, ReferenceOrderID, ReferenceOrderLineID, TransactionDate, Quantity, ActualCost FROM Production.TransactionHistory", source_conn)
        
         ## merging dataframes
        ProductCostHistory_df = pd.merge(ProductCostHistory_df, ProductListPriceHistory_df, on='ProductID', how='left')
        ProductCostHistory_df = pd.merge(ProductCostHistory_df, TransactionHistory_df, on='ProductID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for ProductCostHistory
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductCostHistory]
        (ProductID, Cost_StartDate, Cost_EndDate, StandardCost, List_StartDate, List_EndDate, ListPrice, TransactionID, ReferenceOrderID, ReferenceOrderLineID, TransactionDate, Quantity, ActualCost)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = ProductCostHistory_df[['ProductID', 'StartDate_x', 'EndDate_x', 'StandardCost', 'StartDate_y', 'EndDate_y', 'ListPrice', 'TransactionID', 'ReferenceOrderID', 'ReferenceOrderLineID', 'TransactionDate', 'Quantity', 'ActualCost']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into ProductCostHistory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Error while connecting to the destination SQL Server: ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'DWUnitedOutdoors.dbo.ProductCostHistory'. (208) (SQLExecDirectW); [42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")


In [115]:
#Production.ProductInventory
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ProductInventory_df = pd.read_sql_query("SELECT ProductID, LocationID, Shelf, Bin, Quantity FROM Production.ProductInventory", source_conn)
        Location_df = pd.read_sql_query("SELECT LocationID, Name, CostRate, Availability FROM Production.Location", source_conn)
        
        ## merging dataframes
        ProductInventory_df = pd.merge(ProductInventory_df, Location_df, on='LocationID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductInventory]
        (ProductID, LocationID, Shelf, Bin, Quantity, Name, CostRate, Availability)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = ProductInventory_df[['ProductID', 'LocationID', 'Shelf', 'Bin', 'Quantity', 'Name', 'CostRate', 'Availability']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into ProductInventory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Error while connecting to the destination SQL Server: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_ProductInventory'. Cannot insert duplicate key in object 'dbo.ProductInventory'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [122]:
#Purchasing.ProductVendor
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ProductVendor_df = pd.read_sql_query("SELECT ProductID, BusinessEntityID, AverageLeadTime, StandardPrice, LastReceiptCost, LastReceiptDate, MinOrderQty, MaxOrderQty, OnOrderQty, UnitMeasureCode FROM Purchasing.ProductVendor", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

ProductVendor_df['OnOrderQty'].fillna(0, inplace=True)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductVendor]
        (ProductID, BusinessEntityID, AverageLeadTime, StandardPrice, LastReceiptCost, LastReceiptDate, MinOrderQty, MaxOrderQty, OnOrderQty, UnitMeasureCode)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        data_to_insert = ProductVendor_df[['ProductID', 'BusinessEntityID', 'AverageLeadTime', 'StandardPrice', 'LastReceiptCost', 'LastReceiptDate', 'MinOrderQty', 'MaxOrderQty', 'OnOrderQty', 'UnitMeasureCode']].values.tolist()
        cursor.executemany(insert_query, data_to_insert)
        destination_conn.commit()
        print("Data inserted successfully into ProductVendor table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Error while connecting to the destination SQL Server: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_ProductVendor'. Cannot insert duplicate key in object 'dbo.ProductVendor'. The duplicate key value is (317). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


In [126]:
#ShoppingcartItem
try:
      with pyodbc.connect(source_conn_str) as source_conn:
          Shoppingcart_df = pd.read_sql_query("SELECT ShoppingCartItemID, ShoppingCartID, Quantity, ProductID, DateCreated from Sales.ShoppingCartItem", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ShoppingCartItem]
        (ShoppingCartItemID, ShoppingCardID, Quantity, ProductID, DateCreated)
        VALUES (?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = Shoppingcart_df[['ShoppingCartItemID', 'ShoppingCartID', 'Quantity', 'ProductID', 'DateCreated']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into ShoppingCartItem table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into ShoppingCartItem table.


In [127]:
##Orders
try:
        with pyodbc.connect(source_conn_str) as source_conn:
            Orders_df = pd.read_sql_query("SELECT OrderID, CustomerID, EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry  FROM Orders", source_conn)
            OrdersDetail_df = pd.read_sql_query("SELECT OrderID, ProductID, UnitPrice, Quantity, Discount FROM OrderDetails", source_conn)
            
            ##mergingOrderDetails
            Orders_df = pd.merge(Orders_df, OrdersDetail_df, on='OrderID', how='left')
            
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[OrderDetails]
        (OrderID, CustomerID, EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry, ProductID, UnitPrice, Quantity, Discount)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = Orders_df[['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'Freight', 'ShipName', 'ShipAddress', 'ShipCity', 'ShipRegion', 'ShipPostalCode', 'ShipCountry', 'ProductID', 'UnitPrice', 'Quantity', 'Discount']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into OrderDetails table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

DatabaseError: Execution failed on sql 'SELECT OrderID, CustomerID, EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry  FROM Orders': ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'Orders'. (208) (SQLExecDirectW)")

In [23]:
#unitmeasure
# Establish connection to the source SQL Server database and read
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        unitmeasure_df = pd.read_sql_query("SELECT UnitMeasureCode, Name FROM Production.UnitMeasure", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[UnitMeasure]
        (UnitMeasureCode, Name)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = unitmeasure_df[['UnitMeasureCode', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [37]:
# Fetch data from source
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        bill_of_material_df = pd.read_sql_query("""SELECT BillOfMaterialsID, ProductAssemblyID, ComponentID, StartDate, EndDate, UnitMeasureCode, BOMlevel, PerAssemblyQty FROM Production.BillOfMaterials""", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Validate and clean date columns
date_columns = ['StartDate', 'EndDate']
for column in date_columns:
    bill_of_material_df[column] = pd.to_datetime(bill_of_material_df[column], errors='coerce')

# Drop rows with invalid StartDate
bill_of_material_df.dropna(subset=['StartDate'], inplace=True)

# Replace NaT in EndDate with None
bill_of_material_df['EndDate'] = bill_of_material_df['EndDate'].apply(lambda x: None if pd.isna(x) else x)

# Convert datetime columns to strings in 'YYYY-MM-DD' format, or None
bill_of_material_df['StartDate'] = bill_of_material_df['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d') if not pd.isna(x) else None)
bill_of_material_df['EndDate'] = bill_of_material_df['EndDate'].apply(lambda x: x.strftime('%Y-%m-%d') if not pd.isna(x) else None)

# Ensure PerAssemblyQty is of type float
bill_of_material_df['PerAssemblyQty'] = bill_of_material_df['PerAssemblyQty'].astype(float)

# Handle missing and infinite values in ProductAssemblyID
bill_of_material_df['ProductAssemblyID'] = bill_of_material_df['ProductAssemblyID'].fillna(0).astype(int)

# Print data types of each column
print("Data types of DataFrame columns:")
print(bill_of_material_df.dtypes)

# Insert data into destination
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[BillOfMaterial]
        (BillOfMaterialsID, ProductAssemblyID, ComponentID, StartDate, EndDate, UnitMeasureCode, BOMlevel, PerAssemblyQty)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = bill_of_material_df[
            ['BillOfMaterialsID', 'ProductAssemblyID', 'ComponentID', 'StartDate', 'EndDate', 'UnitMeasureCode', 'BOMlevel', 'PerAssemblyQty']
        ].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into BillOfMaterial table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data types of DataFrame columns:
BillOfMaterialsID      int64
ProductAssemblyID      int32
ComponentID            int64
StartDate             object
EndDate               object
UnitMeasureCode       object
BOMlevel               int64
PerAssemblyQty       float64
dtype: object
Data inserted successfully into BillOfMaterial table.


,BillOfMaterialsID,ProductAssemblyID,ComponentID,StartDate,EndDate,UnitMeasureCode,BOMlevel,PerAssemblyQty
0,893,0,749,2010-05-26,None,EA,0,1.0
1,271,0,750,2010-03-04,2010-05-03,EA,0,1.0
2,34,0,750,2010-05-04,None,EA,0,1.0
3,830,0,751,2010-05-26,None,EA,0,1.0
4,2074,0,752,2010-07-08,None,EA,0,1.0
...,...,...,...,...,...,...,...,...
2674,3063,999,948,2010-12-15,None,EA,1,1.0
2675,75,999,949,2010-03-04,None,EA,1,1.0
2676,285,999,952,2010-03-04,2010-05-03,EA,1,2.0
2677,74,999,952,2010-05-04,None,EA,1,1.0


In [65]:
try:
    with pyodbc.connect(source_conn_str_northwind) as source_conn:
        NorthWindProducts_df = pd.read_sql_query("SELECT * FROM dbo.Products", source_conn)
        NorthWindCategories_df = pd.read_sql_query("SELECT * FROM dbo.Categories", source_conn)
        #merge on CategoryID
        NorthWindSuppliers_df = pd.read_sql_query("SELECT SupplierID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax FROM dbo.Suppliers", source_conn)  
        #merge on SupplierID
        NorthWindProducts_df = pd.merge(NorthWindProducts_df, NorthWindCategories_df, on='CategoryID', how='left')
        NorthWindProducts_df = pd.merge(NorthWindProducts_df, NorthWindSuppliers_df, on='SupplierID', how='left')

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(source_conn_str_northwind) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Product]
        (UnitMeasureCode, Name)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = unitmeasure_df[['UnitMeasureCode', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,...,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,False,...,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,None,EC1 4SD,UK,(171) 555-2222,None
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,False,...,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,None,EC1 4SD,UK,(171) 555-2222,None
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,False,...,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,None,EC1 4SD,UK,(171) 555-2222,None
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,False,...,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,None
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,True,...,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,False,...,Svensk Sjöföda AB,Michael Björn,Sales Representative,Brovallavägen 231,Stockholm,None,S-123 45,Sweden,08-123 45 67,None
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,False,...,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,None,100,Japan,(03) 3555-5011,None
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,False,...,Plutzer Lebensmittelgroßmärkte AG,Martin Bein,International Marketing Mgr.,Bogenallee 51,Frankfurt,None,60439,Germany,(069) 992755,None
75,76,Lakkalikööri,23,1,500 ml,18.00,57,0,20,False,...,Karkki Oy,Anne Heikkonen,Product Manager,Valtakatu 12,Lappeenranta,None,53120,Finland,(953) 10956,None
